In [1]:
import pandas as pd
from datetime import datetime, timedelta
import plotly.graph_objects as go
from statsmodels.tsa.statespace.sarimax import SARIMAX
from glob import glob
import re
%pylab inline

Populating the interactive namespace from numpy and matplotlib


/home/javier/anaconda3/lib/python3.7/site-packages/IPython/core/magics/pylab.py:160: UserWarning:

pylab import has clobbered these variables: ['datetime']
`%matplotlib` prevents importing * from pylab and numpy



In [5]:
#EXTRAER LOS NOMBRES DE TODOS LOS ARCHIVOS PARA SU POSTERIOR LECTURA Y ORDENARLOS EN ORDEN CRONOLÓGICO
archivos = glob('/home/javier/REPOS-GH/TFM-Javier_Casado/DATOS/*Semana*.txt')

def atoi(text):
    return int(text) if text.isdigit() else text

def natural_keys(text):

    return [ atoi(c) for c in re.split(r'(\d+)', text) ]

archivos.sort(key=natural_keys)

In [3]:
#CREAMOS LOS CSVs VACÍOS PARA ALMACENAR LOS DFs DE CADENA PARA SARIMAX
data_prueba = pd.DataFrame(columns=['Target'])
data_prueba.to_csv('/home/javier/REPOS-GH/TFM-Javier_Casado/SARIMAX/atres.csv')
data_prueba.to_csv('/home/javier/REPOS-GH/TFM-Javier_Casado/SARIMAX/sexta.csv')
data_prueba.to_csv('/home/javier/REPOS-GH/TFM-Javier_Casado/SARIMAX/neox.csv')
data_prueba.to_csv('/home/javier/REPOS-GH/TFM-Javier_Casado/SARIMAX/nova.csv')
data_prueba.to_csv('/home/javier/REPOS-GH/TFM-Javier_Casado/SARIMAX/mega.csv')
data_prueba.to_csv('/home/javier/REPOS-GH/TFM-Javier_Casado/SARIMAX/atreseries.csv')
data_prueba.to_csv('/home/javier/REPOS-GH/TFM-Javier_Casado/SARIMAX/telecinco.csv')
data_prueba.to_csv('/home/javier/REPOS-GH/TFM-Javier_Casado/SARIMAX/cuatro.csv')
data_prueba.to_csv('/home/javier/REPOS-GH/TFM-Javier_Casado/SARIMAX/divinity.csv')
data_prueba.to_csv('/home/javier/REPOS-GH/TFM-Javier_Casado/SARIMAX/energy.csv')
data_prueba.to_csv('/home/javier/REPOS-GH/TFM-Javier_Casado/SARIMAX/fdf.csv')
data_prueba.to_csv('/home/javier/REPOS-GH/TFM-Javier_Casado/SARIMAX/bemad.csv')
data_prueba.to_csv('/home/javier/REPOS-GH/TFM-Javier_Casado/SARIMAX/dmax.csv')
data_prueba.to_csv('/home/javier/REPOS-GH/TFM-Javier_Casado/SARIMAX/dkiss.csv')
data_prueba.to_csv('/home/javier/REPOS-GH/TFM-Javier_Casado/SARIMAX/paramount.csv')
data_prueba.to_csv('/home/javier/REPOS-GH/TFM-Javier_Casado/SARIMAX/ten.csv')
data_prueba.to_csv('/home/javier/REPOS-GH/TFM-Javier_Casado/SARIMAX/trece.csv')

In [6]:
for arch in archivos: 
    print(arch)
    #pd.set_option('display.max_columns', None)
    #pd.set_option('display.max_rows', None)
    df = pd.read_csv(arch,error_bad_lines=False,encoding = 'unicode_escape', 
                     header=3, sep='\t', skiprows=[4], skipinitialspace=True, 
                     usecols=['Título','Título/Descripción', 'Cadena', 'Fecha', 'Género','Público objetivo', 
                              'Productora','Calificación de Edad','Ind. 4+ (inv)','Amas de Casa (c/inv.)'
                              ,'Ind. 4 - 12 (c/inv.)', 'Ind. 16+ (c/inv.)','Ind. 4+ (inv).1','Amas de Casa (c/inv.).1',
                              'Ind. 4 - 12 (c/inv.).1', 'Ind. 16+ (c/inv.).1'])

    #ELIMINAMOS LAS CABECERAS DE LOS PROGRAMAS EN LA COLUMNA TÍTULO
    df.dropna(inplace = True)
    df = df[df.Título.str.startswith('<<')]
    df.reset_index(drop = True,inplace = True)

    #ELIMINAMOS LOS SIMBOLOS <> DEL DATA FRAME
    df1 = pd.DataFrame()
    for i in df.columns:
        df1 = pd.concat([df1,df.loc[:,str(i)].str.translate({ord(x): None for x in ('><')})], axis = 1)

    #DIVIDIMOS LA COLUMNA EN DOS PARA PODER OPERAR SOLO CON LA HORA 
    #(NO SE PUEDE PASAR A DATETIME YA QUE NO ESTÁ EN EL RANGO ADECUADO)
    df1[['Hora','Minuto']] = df1.Título.str.split(':', expand = True)
    #df1.head(5)

    #PASAMOS LA COLUMNA FECHA A DATETIME PARA PODER RESTAR DIAS CUANDO SEA NECESARIO
    df1['Fecha'] = pd.to_datetime(df1['Fecha'],format = "%d/%m/%Y", dayfirst=True)
    #df1.dtypes

    #SE CAMBIA EL TIPO DE DATO EN LAS COLUMNAS HORA MINUTO PARA PODER TRANSFORMAR A RANGO 00:00-23:00(STR-INT)
    df1 = df1.astype({'Hora':'int','Minuto':'int'})
    #df1.dtypes

    #SUMAMOS 1 DÍA A LAS HORAS SUPERIORES A 23
    ind = df1[df1['Hora']>23].index
    df1.loc[ind,'Fecha']= df1.loc[ind,'Fecha'] + timedelta(days=1)

    #CONVERTIMOS PARA TENER RANGO 00-23
    df1.Hora = df1.Hora.replace({24:0,25:1,26:2})

    #SE SUSTITUYE EL 24 POR 0 PARA TENER RANGO 00-23
    #df1.Hora = df1.Hora.replace(24,0)


    #CONVIERTO LA FECHA A STRING PARA PODER UNIRLO CON LA HORA
    df1['Fecha']=df1.Fecha.apply(lambda x:str(x.date()))
    df1[df1['Cadena']=='La1'].head(1413)

    #VOLVEMOS A CONVERTIR HORA Y MINUTO A STR PARA PODER UNIRLOS CON ':' COMO SEPARADOR
    df1 = df1.astype({'Hora':'str', 'Minuto':'str'})

    #UNIMOS HORA Y MINUTO PARA DESPUÉS UNIRLO A LA FECHA Y CONVERTIRLO EN TIMESTAMP
    df1 ['Tiempo'] = df1[['Hora', 'Minuto']].apply(lambda x: ':'.join(x), axis=1)
    df1 ['Fecha_comp'] = df1[['Fecha', 'Tiempo']].apply(lambda x: ' '.join(x), axis=1)
    df1.Fecha_comp = pd.to_datetime(df1.Fecha_comp)

    #ELIMINAMOS LAS COLUMNAS GENERADAS Y USADAS PARA LAS TRANSFORMACIONES
    df1 = df1.drop(['Tiempo','Hora', 'Minuto','Fecha','Título'], axis = 1)
    #,'Descripción'

    #CAMBIO COMA POR PUNTO PARA PODER CONVERTIR A ARRAY DE FLOAT
    column = ['Ind. 4+ (inv)','Amas de Casa (c/inv.)', 'Ind. 4 - 12 (c/inv.)','Ind. 16+ (c/inv.)',
              'Ind. 4+ (inv).1','Amas de Casa (c/inv.).1','Ind. 4 - 12 (c/inv.).1', 'Ind. 16+ (c/inv.).1']
    for i in column:
        df1[i]=df1[i].str.replace(',','.')

    #CREO LA VARIABLE TARGET CON LAS COLUMNAS NECESARIAS
    df1['Target'] = df1['Ind. 4+ (inv).1'].apply(lambda x : float(x))
    #df1['Target']= df1[column].apply(lambda x:'/'.join(x),axis=1)
    #df1['Target']= df1.Target.str.split('/').apply(lambda x : [float(i) for i in x])
    #df1['Target']= df1.Target.str.split('/').apply(lambda x:np.array(x,dtype=np.float32))

    #ELIMINAMOS LAS VARIABLES UTILIZADAS EN TARGET
    df1 = df1.drop(column,axis = 1)

    #ELIMINAMOS TITULO/DESCRIPCION DE MOMENTO (!)
    #df1 = df1.drop('Título/Descripción',axis = 1)

    #ORDENO LAS COLUMNAS DE MI DATA FRAME (!) 
    df1 = df1[['Fecha_comp','Título/Descripción','Cadena', 'Género', 'Público objetivo', 'Productora',
           'Calificación de Edad', 'Target']]

    #ESCOJO LAS CADENAS DE INTERÉS
    cadenas_v = ['A3','LA SEXTA','NEOX','NOVA','MEGA','ATRESERIES','T5','CUATRO','DIVINITY','ENERGY','FDF','BEMADtv'
                 ,'DMAX','DKISS','PARAMOUNT NETWORK','TEN','TRECE']
    df1 = df1[df1['Cadena'].isin(cadenas_v)]



    #DATAFRAME OBJETIVO PARA MODELO SARIMAX
    #escogemos las columnas para copiar
    pr = df1[['Fecha_comp','Cadena','Target']]

    #SEPARAMOS DATAFRAMES EN CADENAS PARA DESARROLLAR LOS ALGORITMOS PERTINENTES PARA CADA UNO

    grupos = pr.groupby('Cadena')

    dfatrs = grupos.get_group('A3').groupby('Fecha_comp')['Target'].max().to_frame()
    dfsxt = grupos.get_group('LA SEXTA').groupby('Fecha_comp')['Target'].max().to_frame()
    dfnx = grupos.get_group('NEOX').groupby('Fecha_comp')['Target'].max().to_frame()
    dfnv = grupos.get_group('NOVA').groupby('Fecha_comp')['Target'].max().to_frame()
    dfmg = grupos.get_group('MEGA').groupby('Fecha_comp')['Target'].max().to_frame()
    dfatrser = grupos.get_group('ATRESERIES').groupby('Fecha_comp')['Target'].max().to_frame()
    dftcnc = grupos.get_group('T5').groupby('Fecha_comp')['Target'].max().to_frame()
    dfctr = grupos.get_group('CUATRO').groupby('Fecha_comp')['Target'].max().to_frame()
    dfdvnt = grupos.get_group('DIVINITY').groupby('Fecha_comp')['Target'].max().to_frame()
    dfnrg = grupos.get_group('ENERGY').groupby('Fecha_comp')['Target'].max().to_frame()
    dffdf = grupos.get_group('FDF').groupby('Fecha_comp')['Target'].max().to_frame()
    dfbmd = grupos.get_group('BEMADtv').groupby('Fecha_comp')['Target'].max().to_frame()
    dfdmx = grupos.get_group('DMAX').groupby('Fecha_comp')['Target'].max().to_frame()
    dfdks = grupos.get_group('DKISS').groupby('Fecha_comp')['Target'].max().to_frame()
    dfprmnt = grupos.get_group('PARAMOUNT NETWORK').groupby('Fecha_comp')['Target'].max().to_frame()
    dftn = grupos.get_group('TEN').groupby('Fecha_comp')['Target'].max().to_frame()
    dftrc = grupos.get_group('TRECE').groupby('Fecha_comp')['Target'].max().to_frame()
    
    
    #GUARDAMOS LOS DF DE CADENA EN ARCHIVOS CSV CREADOS PREVIAMENTE PARA CONCATENAR EL PROCESADO DE LOS DATOS
    dfatrs.to_csv('/home/javier/REPOS-GH/TFM-Javier_Casado/SARIMAX/atres.csv', mode = 'a', header = False)
    dfsxt.to_csv('/home/javier/REPOS-GH/TFM-Javier_Casado/SARIMAX/sexta.csv', mode = 'a', header = False)
    dfnx.to_csv('/home/javier/REPOS-GH/TFM-Javier_Casado/SARIMAX/neox.csv', mode = 'a', header = False)
    dfnv.to_csv('/home/javier/REPOS-GH/TFM-Javier_Casado/SARIMAX/nova.csv', mode = 'a', header = False)
    dfmg.to_csv('/home/javier/REPOS-GH/TFM-Javier_Casado/SARIMAX/mega.csv', mode = 'a', header = False)
    dfatrser.to_csv('/home/javier/REPOS-GH/TFM-Javier_Casado/SARIMAX/atreseries.csv', mode = 'a', header = False)
    dftcnc.to_csv('/home/javier/REPOS-GH/TFM-Javier_Casado/SARIMAX/telecinco.csv', mode = 'a', header = False)
    dfctr.to_csv('/home/javier/REPOS-GH/TFM-Javier_Casado/SARIMAX/cuatro.csv', mode = 'a', header = False)
    dfdvnt.to_csv('/home/javier/REPOS-GH/TFM-Javier_Casado/SARIMAX/divinity.csv', mode = 'a', header = False)
    dfnrg.to_csv('/home/javier/REPOS-GH/TFM-Javier_Casado/SARIMAX/energy.csv', mode = 'a', header = False)
    dffdf.to_csv('/home/javier/REPOS-GH/TFM-Javier_Casado/SARIMAX/fdf.csv', mode = 'a', header = False)
    dfbmd.to_csv('/home/javier/REPOS-GH/TFM-Javier_Casado/SARIMAX/bemad.csv', mode = 'a', header = False)
    dfdmx.to_csv('/home/javier/REPOS-GH/TFM-Javier_Casado/SARIMAX/dmax.csv', mode = 'a', header = False)
    dfdks.to_csv('/home/javier/REPOS-GH/TFM-Javier_Casado/SARIMAX/dkiss.csv', mode = 'a', header = False)
    dfprmnt.to_csv('/home/javier/REPOS-GH/TFM-Javier_Casado/SARIMAX/paramount.csv', mode = 'a', header = False)
    dftn.to_csv('/home/javier/REPOS-GH/TFM-Javier_Casado/SARIMAX/ten.csv', mode = 'a', header = False)
    dftrc.to_csv('/home/javier/REPOS-GH/TFM-Javier_Casado/SARIMAX/trece.csv', mode = 'a', header = False)
    
    print('ok')


/home/javier/REPOS-GH/TFM-Javier_Casado/DATOS/2019 - Semana 17 (28042019).txt
ok
/home/javier/REPOS-GH/TFM-Javier_Casado/DATOS/2019 - Semana 18 (05052019).txt
ok
/home/javier/REPOS-GH/TFM-Javier_Casado/DATOS/2019 - Semana 19 (12052019).txt
ok
/home/javier/REPOS-GH/TFM-Javier_Casado/DATOS/2019 - Semana 20 (19052019).txt
ok
/home/javier/REPOS-GH/TFM-Javier_Casado/DATOS/2019 - Semana 21 (26052019).txt
ok
/home/javier/REPOS-GH/TFM-Javier_Casado/DATOS/2019 - Semana 22 (02062019).txt
ok
/home/javier/REPOS-GH/TFM-Javier_Casado/DATOS/2019 - Semana 23 (09062019).txt
ok
/home/javier/REPOS-GH/TFM-Javier_Casado/DATOS/2019 - Semana 24 (16062019).txt
ok
/home/javier/REPOS-GH/TFM-Javier_Casado/DATOS/2019 - Semana 25 (23062019).txt
ok
/home/javier/REPOS-GH/TFM-Javier_Casado/DATOS/2019 - Semana 26 (30062019).txt
ok
/home/javier/REPOS-GH/TFM-Javier_Casado/DATOS/2020 - Semana 1 (05012020).txt
ok
/home/javier/REPOS-GH/TFM-Javier_Casado/DATOS/2020 - Semana 2 (12012020).txt
ok
/home/javier/REPOS-GH/TFM-Javi

In [7]:
df = pd.read_csv('/home/javier/REPOS-GH/TFM-Javier_Casado/SARIMAX/atres.csv', index_col=0)

In [11]:
plt.figure(figsize=(18,10))
plt.plot(df)

Error in callback <function install_repl_displayhook.<locals>.post_execute at 0x7f48b15127a0> (for post_execute):


KeyboardInterrupt: 

Error in callback <function flush_figures at 0x7f48b144e170> (for post_execute):


KeyboardInterrupt: 

In [9]:
df.head(10)

,Target
2018-01-01 02:30:00,921.306
2018-01-01 02:31:00,928.767
2018-01-01 02:32:00,870.787
2018-01-01 02:33:00,906.876
2018-01-01 02:34:00,956.670
2018-01-01 02:35:00,928.635
2018-01-01 02:36:00,912.941
2018-01-01 02:37:00,940.245
2018-01-01 02:38:00,872.803
2018-01-01 02:39:00,893.625


In [10]:
df.tail(10)

,Target
2020-03-16 02:20:00,150.987
2020-03-16 02:21:00,150.987
2020-03-16 02:22:00,153.091
2020-03-16 02:23:00,153.091
2020-03-16 02:24:00,155.293
2020-03-16 02:25:00,146.432
2020-03-16 02:26:00,157.593
2020-03-16 02:27:00,157.593
2020-03-16 02:28:00,153.002
2020-03-16 02:29:00,150.688
